In [1]:
import geopandas as gp
import pandas as pd
import numpy as np
from src.constants import features_for_total
from src.ftr_aggregate import *
%matplotlib inline
from IPython.display import display

In [ ]:
from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
init_notebook_mode(True)

In [5]:
features = load_separate_features()

loading feature: feature_seg_attribute_dc.csv
loading feature: feature_crash_dc.csv
loading feature: feature_311_dc.csv
loading feature: feature_parking_violations_dc.csv
loading feature: feature_vision0_dc.csv
loading feature: feature_mov_violations_dc.csv
loading feature: feature_poi_dc.csv
loading feature: feature_bk_facs_dc.csv
loading feature: feature_seg_as_edge_dc.csv
loading feature: feature_seg_as_node_dc.csv
loading feature: feature_crime_dc.csv
loading feature: feature_bk_opendc_dc.csv


# plot map

In [18]:
from src.leaflet_creation import create_map_visualization, gradient_color

In [11]:
seg = gp.read_file('data/segments_dc.geojson')

In [143]:
def min_max(arr, mini=None, maxi=None):
    if mini is None:
        mini = arr.min()
    if maxi is None:
        maxi = arr.max()
    return (arr - mini)/(maxi-mini)

In [150]:
mini = 0
maxi = f311_total['log'].max()

In [151]:
f311_total = seg[['geometry']].copy()
f311_total['f311_total'] = features['311'].drop(['YEAR', 'MONTH'],axis=1).groupby(level=0).mean()['311_total']
f311_total.dropna(inplace=True)
f311_total = f311_total[f311_total.f311_total!=0].copy()
f311_total['log'] = np.log(f311_total.f311_total*100)
f311_total['color'] = min_max(f311_total['log'], mini, maxi).apply(gradient_color)

In [152]:
f311_road = seg[['geometry']].copy()
f311_road['f311_road'] = features['311'].drop(['YEAR', 'MONTH'],axis=1).groupby(level=0).mean()['311_ROADWAY REPAIR']
f311_road.dropna(inplace=True)
f311_road = f311_road[f311_road.f311_road!=0].copy()
f311_road['log'] = np.log(f311_road.f311_road*100)
f311_road['color'] = min_max(f311_road['log'], mini, maxi).apply(gradient_color)

In [153]:
f311_potholes = seg[['geometry']].copy()
f311_potholes['f311_potholes'] = features['311'].drop(['YEAR', 'MONTH'],axis=1).groupby(level=0).mean()['311_POTHOLES']
f311_potholes.dropna(inplace=True)
f311_potholes = f311_potholes[f311_potholes.f311_potholes!=0].copy()
f311_potholes['log'] = np.log(f311_potholes.f311_potholes*100)
f311_potholes['color'] = min_max(f311_potholes['log'], mini, maxi).apply(gradient_color)

In [154]:
html_title = 'feature map'
file_path = ''
file_name = 'feature map'
lon, lat = -77.0908494, 38.9045525  #D.C.
zoom = 12
init_layers = ['streets', 'stsg']
map_layers = ['light','streets', 'satellite']
binding_data=[['f311_total_map','total'],['f311_potholes_map','potholes'],['f311_road_map', 'road']]
gpdfs = [f311_total, f311_potholes, f311_road]
create_map_visualization(html_title, file_path, file_name, lat, lon, zoom, init_layers, map_layers, binding_data, gpdfs, width=1000)

# plot time

In [4]:
data = []
for name, ftr in features.items():
    if 'YEAR' in ftr.columns:
        new_ftr = ftr.copy()
        new_ftr = filter_year(new_ftr, years=range(2008, 2018), keep_year=True)
        new_ftr = filter_total(new_ftr, how="TOTAL")
        stat = new_ftr.groupby(['YEAR','MONTH']).sum()
        stat.index = ['{}_{}'.format(int(x), int(y)) for x,y in stat.index.tolist()]
        for x in stat.columns:
            data.append(go.Scatter(y=stat[x], x=stat.index, name=name+x))

In [5]:
iplot(data)

# plot each variable

In [1]:
for name, df in features.items():
    ftr = df.copy()
    ftr = filter_year(ftr)
    if name in features_for_total:
        ftr = filter_total(ftr, name, how="NO_TOTAL")
    # ftr,mapping = encode_col(ftr,'crime')

    stat = ftr.groupby(level=0).sum()
    print '='*20, name, stat.shape
    top_20 = stat.astype(bool).sum().sort_values(ascending=False).index[:20]
    data = [go.Box(y=stat[x], name = x) for x in top_20]
    layout = go.Layout(title=name+str(stat.shape))
    fig = go.Figure(data=data, layout=layout)
    iplot(fig)
#     break
    
#     if stat.shape[1]<=20:
#         data = [go.Box(y=stat[x], name = x) for x in stat.columns]
#         layout = go.Layout(title=name+str(stat.shape))
#         fig = go.Figure(data=data, layout=layout)

#         iplot(fig)
        